Question 1:
In wastewater treatment, oxygen is supplied to support biological processes in an aeration tank. The oxygen transfer rate (OTR) is crucial for maintaining sufficient dissolved oxygen (DO) levels to meet biological oxygen demand (BOD).
Develop a python-based iterative app for a common problem in wastewater treatment: calculating the oxygen transfer rate (OTR) in an aeration tank. This process involves iteratively calculating the dissolved oxygen (DO) levels based on oxygen transfer and consumption rates.
This app will:
•	Simulate the dissolved oxygen (DO) profile over time in the aeration tank.
•	Iteratively calculate DO levels using the oxygen transfer and consumption rates.
•	Allow users to adjust parameters such as oxygen transfer efficiency (OTE), oxygen consumption rate, and initial DO.
•	Plot the DO concentration over time.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def simulate_DO_profile(OTE, oxygen_consumption_rate, initial_DO, time_duration, time_step):
    # Constants
    oxygen_saturation = 9.0  # mg/L, typical saturation level of DO in water at 20°C
    transfer_rate_constant = OTE * oxygen_saturation

    # Time simulation
    time = np.arange(0, time_duration, time_step)
    DO_levels = np.zeros(len(time))
    DO_levels[0] = initial_DO

    # Iterative calculation of DO levels over time
    for i in range(1, len(time)):
        oxygen_transfer = transfer_rate_constant * (oxygen_saturation - DO_levels[i-1]) * time_step
        oxygen_consumption = oxygen_consumption_rate * DO_levels[i-1] * time_step
        DO_levels[i] = DO_levels[i-1] + oxygen_transfer - oxygen_consumption
        DO_levels[i] = max(DO_levels[i], 0)  # Ensure DO levels don't go below zero

    # Plotting the DO concentration over time
    plt.figure(figsize=(10, 6))
    plt.plot(time, DO_levels, label="DO Concentration", color='blue')
    plt.xlabel("Time (minutes)")
    plt.ylabel("DO Concentration (mg/L)")
    plt.title("Dissolved Oxygen (DO) Profile in Aeration Tank")
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.show()

# Interactive sliders to adjust parameters
interact(
    simulate_DO_profile,
    OTE=FloatSlider(value=0.3, min=0.1, max=0.6, step=0.01, description='OTE'),
    oxygen_consumption_rate=FloatSlider(value=0.02, min=0.01, max=0.1, step=0.01, description='Consumption Rate'),
    initial_DO=FloatSlider(value=2.0, min=0.0, max=9.0, step=0.1, description='Initial DO'),
    time_duration=FloatSlider(value=60, min=10, max=120, step=5, description='Time Duration (min)'),
    time_step=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Time Step (min)')
)


interactive(children=(FloatSlider(value=0.3, description='OTE', max=0.6, min=0.1, step=0.01), FloatSlider(valu…

<function __main__.simulate_DO_profile(OTE, oxygen_consumption_rate, initial_DO, time_duration, time_step)>

Question 2:
Liquid – Liquid extraction is useful in the separation of compounds based on their different solubilities in two immiscible liquids. Develop a python-based calculator for a liquid-liquid extraction process, which is a common separation technique in chemical engineering. The app will calculate the equilibrium compositions of two immiscible phases and plot the distribution curve (also known as the tie-line diagram) for a ternary liquid-liquid system.
The app will:
•	Calculate the distribution ratio (D) of a solute between two immiscible solvents.
•	Use a distribution curve to visualize the relationship between the solute concentration in the two phases.
•	Plot a tie-line diagram, showing the equilibrium compositions in both phases across different initial solute concentrations.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def calculate_distribution_curve(K, initial_concentration, solvent_ratio):
    # Define the range of initial solute concentrations in the feed
    solute_concentration_feed = np.linspace(0, initial_concentration, 100)

    # Calculate the equilibrium solute concentrations in both phases
    solute_concentration_phase1 = solute_concentration_feed / (1 + K * solvent_ratio)
    solute_concentration_phase2 = K * solute_concentration_phase1

    return solute_concentration_phase1, solute_concentration_phase2, solute_concentration_feed

def plot_tie_line_diagram(K, initial_concentration, solvent_ratio):
    # Calculate equilibrium solute concentrations
    phase1, phase2, feed = calculate_distribution_curve(K, initial_concentration, solvent_ratio)

    # Plotting the tie-line diagram
    plt.figure(figsize=(8, 6))
    plt.plot(phase1, phase2, label="Tie-Line", color='green')
    plt.scatter(phase1, phase2, color='red', s=10, label="Equilibrium Points")
    plt.xlabel("Solute Concentration in Phase 1 (mg/L)")
    plt.ylabel("Solute Concentration in Phase 2 (mg/L)")
    plt.title("Tie-Line Diagram for Liquid-Liquid Extraction")
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.show()

# Interactive sliders to adjust parameters
interact(
    plot_tie_line_diagram,
    K=FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Distribution Ratio (K)'),
    initial_concentration=FloatSlider(value=100, min=10, max=500, step=10, description='Initial Conc. (mg/L)'),
    solvent_ratio=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Solvent Ratio')
)

interactive(children=(FloatSlider(value=2.0, description='Distribution Ratio (K)', max=10.0, min=0.1), FloatSl…

<function __main__.plot_tie_line_diagram(K, initial_concentration, solvent_ratio)>

Question 3:
Distillation is a key separation technique used in chemical engineering to separate liquid mixtures based on differences in their volatilities. Develop a Python-based calculator for the design and performance evaluation of a distillation column. This app will calculate the number of theoretical stages required for separating a binary mixture using the McCabe-Thiele method and plot the corresponding equilibrium curve.
The app will:
•	Calculate the minimum number of stages required for the separation of a binary mixture given the feed composition, reflux ratio, and product specifications.
•	Plot the equilibrium curve and operating lines (rectifying and stripping sections) on an x-y diagram, showing the relationship between the liquid and vapor compositions in the distillation column.
•	Draw the McCabe-Thiele diagram to visualize the number of theoretical stages and the stepping process between equilibrium and operating lines.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def equilibrium_curve(x):
    # Example equilibrium curve for a binary mixture (e.g., using a typical relative volatility)
    alpha = 2.5  # Relative volatility
    y = (alpha * x) / (1 + (alpha - 1) * x)
    return y

def operating_line(x, reflux_ratio, xD, xB, q_line=False, feed_composition=0.5):
    if q_line:
        # Equation for the q-line (feed line)
        q = 0.9  # Example value; you can adjust q based on the feed condition
        if q == 1.0:
            # Special case for q = 1 (saturated liquid feed), where the q-line is vertical
            return np.full_like(x, feed_composition)
        else:
            return (q / (q - 1)) * x - feed_composition / (q - 1)
    else:
        # Rectifying section: y = (R/(R+1)) * x + xD/(R+1)
        return (reflux_ratio / (reflux_ratio + 1)) * x + xD / (reflux_ratio + 1)

def mccabe_thiele_diagram(feed_composition, xD, xB, reflux_ratio):
    x = np.linspace(0, 1, 100)
    y_eq = equilibrium_curve(x)

    # Plotting the equilibrium curve
    plt.figure(figsize=(10, 8))
    plt.plot(x, y_eq, label="Equilibrium Curve", color='blue')
    plt.plot(x, x, label="y = x (45° line)", color='black', linestyle='--')

    # Plotting the operating lines
    y_rectifying = operating_line(x, reflux_ratio, xD, xB)
    y_stripping = operating_line(x, reflux_ratio, xD, xB, q_line=True)

    plt.plot(x, y_rectifying, label="Rectifying Line", color='red')
    plt.plot(x, y_stripping, label="Stripping Line (q-line)", color='green')

    # Drawing the McCabe-Thiele diagram (stepping off stages)
    x_stage = [feed_composition]
    y_stage = [feed_composition]

    x_current = feed_composition
    while x_current < xD and x_current > xB:
        # Move up to the equilibrium curve
        y_current = equilibrium_curve(x_current)
        x_stage.append(x_current)
        y_stage.append(y_current)

        # Move horizontally to the operating line
        x_current = y_current
        y_current = operating_line(x_current, reflux_ratio, xD, xB)
        x_stage.append(x_current)
        y_stage.append(y_current)

    plt.plot(x_stage, y_stage, marker='o', color='purple', label="Theoretical Stages")

    # Labels and legend
    plt.xlabel("Liquid Composition x")
    plt.ylabel("Vapor Composition y")
    plt.title("McCabe-Thiele Diagram for Binary Distillation")
    plt.legend()
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    plt.show()

# Interactive sliders to adjust parameters
interact(
    mccabe_thiele_diagram,
    feed_composition=FloatSlider(value=0.5, min=0.1, max=0.9, step=0.01, description='Feed Composition'),
    xD=FloatSlider(value=0.95, min=0.5, max=1.0, step=0.01, description='Distillate Composition xD'),
    xB=FloatSlider(value=0.05, min=0.0, max=0.5, step=0.01, description='Bottoms Composition xB'),
    reflux_ratio=FloatSlider(value=2.0, min=1.0, max=10.0, step=0.1, description='Reflux Ratio')
)

interactive(children=(FloatSlider(value=0.5, description='Feed Composition', max=0.9, min=0.1, step=0.01), Flo…

<function __main__.mccabe_thiele_diagram(feed_composition, xD, xB, reflux_ratio)>

Question 4:
Chemical reactors are central to chemical engineering processes, where chemical reactions occur under controlled conditions. Develop a Python-based calculator for designing an isothermal Continuous Stirred Tank Reactor (CSTR) and a Plug Flow Reactor (PFR) for a first-order irreversible reaction. This app will calculate the reactor volume required to achieve a specified conversion and compare the performance of CSTR and PFR reactors.
The app will:
•	Calculate the reactor volume (V) for both CSTR and PFR given the feed rate, rate constant, and target conversion.
•	Compare the performance of CSTR and PFR, showing the volume required to achieve the same conversion in both reactors.
•	Plot the conversion profile for a PFR along the reactor length and the conversion as a function of reactor volume for the CSTR.
•	Allow the user to input reaction rate constants, flow rates, and initial concentrations, and dynamically adjust the conversion and reactor volumes.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def reactor_volumes(rate_constant, flow_rate, initial_concentration, conversion):
    # Calculate CSTR volume: V = (F / k) * (X / (1 - X))
    V_CSTR = (flow_rate / rate_constant) * (conversion / (1 - conversion))

    # Calculate PFR volume: V = (F / k) * ln(1 / (1 - X))
    V_PFR = (flow_rate / rate_constant) * np.log(1 / (1 - conversion))

    return V_CSTR, V_PFR

def plot_reactor_performance(rate_constant, flow_rate, initial_concentration, conversion):
    # Calculate volumes
    V_CSTR, V_PFR = reactor_volumes(rate_constant, flow_rate, initial_concentration, conversion)

    # Plot CSTR volume vs. conversion
    conversions = np.linspace(0.01, 0.99, 100)
    V_CSTR_curve = (flow_rate / rate_constant) * (conversions / (1 - conversions))

    plt.figure(figsize=(12, 6))

    # CSTR plot
    plt.subplot(1, 2, 1)
    plt.plot(conversions, V_CSTR_curve, label="CSTR Volume", color="blue")
    plt.scatter([conversion], [V_CSTR], color="red", label=f"V_CSTR = {V_CSTR:.2f} m³", zorder=5)
    plt.xlabel("Conversion (X)")
    plt.ylabel("Reactor Volume (m³)")
    plt.title("CSTR Volume vs. Conversion")
    plt.legend()
    plt.grid(visible=True, linestyle='--', alpha=0.5)

    # PFR plot: Conversion profile along reactor length
    reactor_length = np.linspace(0, V_PFR, 100)
    conversion_profile = 1 - np.exp(-rate_constant * reactor_length / flow_rate)

    plt.subplot(1, 2, 2)
    plt.plot(reactor_length, conversion_profile, label="PFR Conversion Profile", color="green")
    plt.scatter([V_PFR], [conversion], color="red", label=f"V_PFR = {V_PFR:.2f} m³", zorder=5)
    plt.xlabel("Reactor Volume (m³)")
    plt.ylabel("Conversion (X)")
    plt.title("PFR Conversion Profile")
    plt.legend()
    plt.grid(visible=True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

# Interactive sliders to adjust parameters
interact(
    plot_reactor_performance,
    rate_constant=FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Rate Constant (1/s)'),
    flow_rate=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Flow Rate (m³/s)'),
    initial_concentration=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Initial Concentration (mol/m³)'),
    conversion=FloatSlider(value=0.8, min=0.1, max=0.99, step=0.01, description='Target Conversion')
)

interactive(children=(FloatSlider(value=0.1, description='Rate Constant (1/s)', max=1.0, min=0.01, step=0.01),…

<function __main__.plot_reactor_performance(rate_constant, flow_rate, initial_concentration, conversion)>